# Test Matrix
Report the non-default environments from the ReFrame config.

This is very `hpc-tests` specific, as it assumes tests map 1:1 to environments.

In [30]:
import sys, os, json, pprint, re
import pandas as pd

CFG_PATH = '../reframe_config.py'

def read_config(path):
    config = {}
    with open(path) as cfgf:
        exec(cfgf.read(), {}, config)
    return config['site_configuration']

def get_ver_from_spack(modulename):
    """ Hacky, non-robust way to get version from spack module name - works only for csd3 & alaska naming conventions """
    for pat in [r'\w+\/([\d.]+)-\w+', # hpl/2.3-iyor3px
                '([\d.]+)']: # intel-mpi-benchmarks-2019.6-gcc-9.1.0-5tbknir
        s = re.search(pat, modulename)
        if s:
            return s.group(1)
    return modulename
    

matrix = {} # environment (= test) name -> ['system1:partition1', ...]
all_sys_parts = [] # ['system1:partition1', ...]
sys_envs = {}

cfg = read_config(CFG_PATH)
for env in cfg['environments']:
    for ts in env.get('target_systems', []):
        env_name = env['name']
        modules = env.get('modules', [])
        if len(modules) > 1:
            raise ValueError('> 1 module in %s:%s' % (ts, env_name))
        elif len(modules) == 1:
            module = get_ver_from_spack(modules[0])            
        else:
            module = '-'
        if ts not in sys_envs:
            sys_envs[ts] = {}
        if env_name in sys_envs[ts]:
            raise ValueError('2 definitions for %s:%s' % (ts, env_name))
        sys_envs[ts][env_name] = module
        
df = pd.DataFrame(sys_envs).fillna('-')
display(df)
print('CSV output follows:')
df.to_csv()

,alaska:ib-gcc9-openmpi4-ucx,alaska:roce-gcc9-openmpi4-ucx,alaska:ib-gcc9-impi-verbs,alaska:roce-gcc9-impi-verbs,csd3:cclake-ib-gcc9-ompi3-ucx,csd3:cclake-roce-gcc9-ompi3-ucx,csd3:cclake-ib-icc19-impi19-ucx,csd3:cclake-roce-icc19-impi19-ucx
imb,2019.5,2019.5,2019.5,2019.5,2019.6,2019.6,-,-
gromacs,2016.4,2016.4,-,-,2016.6,2016.6,-,-
omb,5.6.2,5.6.2,5.6.2,5.6.2,5.6.3,5.6.3,-,-
hpl,2.3,2.3,-,-,-,-,-,-
openfoam,7,7,-,-,7,7,-,-
cp2k,7.1,7.1,-,-,-,-,-,-
intel-hpl,-,-,2020.1.217,2020.1.217,-,-,-,-
intel-hpcg,-,-,2020.1.217,2020.1.217,-,-,-,-
castep,-,-,-,-,-,-,castep,castep
wrf,-,-,-,-,-,-,-,-


CSV output follows:


',alaska:ib-gcc9-openmpi4-ucx,alaska:roce-gcc9-openmpi4-ucx,alaska:ib-gcc9-impi-verbs,alaska:roce-gcc9-impi-verbs,csd3:cclake-ib-gcc9-ompi3-ucx,csd3:cclake-roce-gcc9-ompi3-ucx,csd3:cclake-ib-icc19-impi19-ucx,csd3:cclake-roce-icc19-impi19-ucx\nimb,2019.5,2019.5,2019.5,2019.5,2019.6,2019.6,-,-\ngromacs,2016.4,2016.4,-,-,2016.6,2016.6,-,-\nomb,5.6.2,5.6.2,5.6.2,5.6.2,5.6.3,5.6.3,-,-\nhpl,2.3,2.3,-,-,-,-,-,-\nopenfoam,7,7,-,-,7,7,-,-\ncp2k,7.1,7.1,-,-,-,-,-,-\nintel-hpl,-,-,2020.1.217,2020.1.217,-,-,-,-\nintel-hpcg,-,-,2020.1.217,2020.1.217,-,-,-,-\ncastep,-,-,-,-,-,-,castep,castep\nwrf,-,-,-,-,-,-,-,-\n'